ResNet-CNN

In [1]:
import os
import random
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity 
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64, blocks=2, stride=1)
        self.layer2 = self._make_layer(64, 128, blocks=2, stride=2)
        self.layer3 = self._make_layer(128, 256, blocks=2, stride=2)
        self.layer4 = self._make_layer(256, 512, blocks=2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(BasicBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = []
        self.transform = transform
        
        df = pd.read_csv(csv_file)
        
        self.class_labels = df['folder_name'].unique()
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.class_labels)}
        
        for idx, row in df.iterrows():
            image_path = row['image_path']
            label = row['folder_name']
            label_idx = self.class_to_idx[label]
            self.data.append((image_path, label_idx))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_csv = "train_images.csv"
train_dataset = CustomDataset(csv_file=train_csv, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = ResNet(num_classes=len(train_dataset.class_labels))
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 21
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

torch.save(model.state_dict(), "resnet.pth")
print(f"Model saved to plant_disease_resnet.pth")


Using device: cpu
Epoch [1/21], Loss: 1.8641, Accuracy: 33.97%
Epoch [2/21], Loss: 1.4669, Accuracy: 50.63%
Epoch [3/21], Loss: 1.2326, Accuracy: 58.78%
Epoch [4/21], Loss: 1.0357, Accuracy: 65.16%
Epoch [5/21], Loss: 0.8605, Accuracy: 71.26%
Epoch [6/21], Loss: 0.6764, Accuracy: 77.60%
Epoch [7/21], Loss: 0.5344, Accuracy: 82.72%
Epoch [8/21], Loss: 0.4210, Accuracy: 85.88%
Epoch [9/21], Loss: 0.3180, Accuracy: 89.87%
Epoch [10/21], Loss: 0.2515, Accuracy: 91.51%
Epoch [11/21], Loss: 0.1908, Accuracy: 93.89%
Epoch [12/21], Loss: 0.1530, Accuracy: 94.66%
Epoch [13/21], Loss: 0.1191, Accuracy: 96.09%
Epoch [14/21], Loss: 0.1269, Accuracy: 95.94%
Epoch [15/21], Loss: 0.0933, Accuracy: 96.81%
Epoch [16/21], Loss: 0.0916, Accuracy: 97.01%
Epoch [17/21], Loss: 0.1036, Accuracy: 96.33%
Epoch [18/21], Loss: 0.0746, Accuracy: 97.60%
Epoch [19/21], Loss: 0.0469, Accuracy: 98.55%
Epoch [20/21], Loss: 0.0588, Accuracy: 97.99%
Epoch [21/21], Loss: 0.0926, Accuracy: 97.17%
Model saved to plant_dise

ViT

In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms
from transformers import AutoModelForImageClassification, AutoImageProcessor
import pandas as pd
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class_labels = [
    'bacterial_leaf_blight',
    'bacterial_leaf_streak',
    'bacterial_panicle_blight',
    'blast',
    'brown_spot',
    'dead_heart',
    'downy_mildew',
    'hispa',
    'normal',
    'tungro'
]

# Data transformations with augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_paths = self.data['image_path'].values
        self.labels = self.data['folder_name'].values
        self.transform = transform
        self.class_labels = [
            'bacterial_leaf_blight',
            'bacterial_leaf_streak',
            'bacterial_panicle_blight',
            'blast',
            'brown_spot',
            'dead_heart',
            'downy_mildew',
            'hispa',
            'normal',
            'tungro'
        ]
        self.label_map = {label: idx for idx, label in enumerate(self.class_labels)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        folder_name = self.labels[idx]
        label = self._get_label_from_folder_name(folder_name)
        
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

    def _get_label_from_folder_name(self, folder_name):
        label_vector = [0] * len(self.class_labels)
        if folder_name in self.label_map:
            label_vector[self.label_map[folder_name]] = 1
        return torch.tensor(label_vector, dtype=torch.float32)

# Initialize processor and model
processor = AutoImageProcessor.from_pretrained("facebook/deit-tiny-patch16-224", use_fast=True, trust_remote_code=True)
model = AutoModelForImageClassification.from_pretrained("facebook/deit-tiny-patch16-224", trust_remote_code=True)
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier.in_features, len(class_labels))
)
model.to(device)

# Dataset and subset loader
csv_file = "train_images.csv"  
full_dataset = CustomImageDataset(csv_file=csv_file, transform=transform)

def get_random_subset(dataset, num_samples_per_class):
    class_indices = {cls: [] for cls in class_labels}
    for idx, (_, label) in enumerate(dataset):
        cls_index = torch.argmax(label).item()
        class_name = class_labels[cls_index]
        class_indices[class_name].append(idx)

    subset_indices = []
    for cls, indices in class_indices.items():
        subset_indices.extend(random.sample(indices, min(len(indices), num_samples_per_class)))

    return Subset(dataset, subset_indices)

subset_dataset = get_random_subset(full_dataset, num_samples_per_class=250)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

num_epochs = 30
best_accuracy = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=images)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        predicted = torch.sigmoid(outputs.logits) > 0.5
        correct += (predicted == labels).all(dim=1).sum().item()
        total += labels.size(0)

    scheduler.step()
    epoch_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_model.pth")
        print("Model checkpoint saved.")

print(f"Training complete. Best accuracy: {best_accuracy:.2f}%")


c:\Users\edwar\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


c:\Users\edwar\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\edwar\.cache\huggingface\hub\models--facebook--deit-tiny-patch16-224. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Epoch [1/30], Loss: 0.3094, Accuracy: 5.76%
Model checkpoint saved.
Epoch [2/30], Loss: 0.2267, Accuracy: 27.52%
Model checkpoint saved.
Epoch [3/30], Loss: 0.1947, Accuracy: 40.24%
Model checkpoint saved.
Epoch [4/30], Loss: 0.1627, Accuracy: 51.04%
Model checkpoint saved.
Epoch [5/30], Loss: 0.1443, Accuracy: 58.08%
Model checkpoint saved.
Epoch [6/30], Loss: 0.1262, Accuracy: 64.76%
Model checkpoint saved.
Epoch [7/30], Loss: 0.1014, Accuracy: 71.92%
Model checkpoint saved.
Epoch [8/30], Loss: 0.0872, Accuracy: 77.28%
Model checkpoint saved.
Epoch [9/30], Loss: 0.0847, Accuracy: 78.68%
Model checkpoint saved.
Epoch [10/30], Loss: 0.0768, Accuracy: 81.08%
Model checkpoint saved.
Epoch [11/30], Loss: 0.0532, Accuracy: 87.44%
Model checkpoint saved.
Epoch [12/30], Loss: 0.0450, Accuracy: 90.12%
Model checkpoint saved.
Epoch [13/30], Loss: 0.0415, Accuracy: 90.48%
Model checkpoint saved.
Epoch [14/30], Loss: 0.0382, Accuracy: 91.32%
Model checkpoint saved.
Epoch [15/30], Loss: 0.0360, A